In [1]:
!pip install -q PyMuPDF
!pip install -q fuzzywuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.2 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.1 MB/s eta 0:00:00


In [4]:
import fitz
from fuzzywuzzy import fuzz, process
import re
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import json

In [5]:
@dataclass
class ParagraphMatch:
    text: str
    page_number: int
    bbox: Tuple[float, float, float, float]  # (x0, y0, x1, y1)
    similarity_score: int
    block_number: int
    line_numbers: List[int]

class PDFParagraphLocator:
    def __init__(self, pdf_path: str, min_similarity: int = 70):
        self.pdf_path = pdf_path
        self.min_similarity = min_similarity
        self.doc = None
        self.paragraphs = []

    def __enter__(self):
        self.doc = fitz.open(self.pdf_path)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.doc:
            self.doc.close()

    def extract_paragraphs(self) -> List[Dict]:
        paragraphs = []
        for page_num in range(len(self.doc)):
            page = self.doc[page_num]
            blocks = page.get_text("dict")
            for block_idx, block in enumerate(blocks["blocks"]):
                if "lines" not in block:
                    continue
                paragraph_text = ""
                line_numbers = []
                bbox = None
                for line_idx, line in enumerate(block["lines"]):
                    line_text = ""
                    for span in line["spans"]:
                        line_text += span["text"]
                    if line_text.strip():
                        paragraph_text += line_text + " "
                        line_numbers.append(line_idx)
                        if bbox is None:
                            bbox = line["bbox"]
                        else:
                            bbox = (
                                min(bbox[0], line["bbox"][0]),  # x0
                                min(bbox[1], line["bbox"][1]),  # y0
                                max(bbox[2], line["bbox"][2]),  # x1
                                max(bbox[3], line["bbox"][3])   # y1
                            )
                paragraph_text = re.sub(r'\s+', ' ', paragraph_text.strip())
                if len(paragraph_text) > 20:
                    paragraphs.append({
                        'text': paragraph_text,
                        'page_number': page_num + 1,
                        'bbox': bbox,
                        'block_number': block_idx,
                        'line_numbers': line_numbers,
                        'char_count': len(paragraph_text),
                        'word_count': len(paragraph_text.split())
                    })
        self.paragraphs = paragraphs
        return paragraphs

    def find_paragraphs(self, search_text: str, max_results: int = 5) -> List[ParagraphMatch]:
        if not self.paragraphs:
            self.extract_paragraphs()
        matches = []
        for para in self.paragraphs:
            similarity = fuzz.partial_ratio(search_text.lower(), para['text'].lower())
            if similarity >= self.min_similarity:
                match = ParagraphMatch(
                    text=para['text'],
                    page_number=para['page_number'],
                    bbox=para['bbox'],
                    similarity_score=similarity,
                    block_number=para['block_number'],
                    line_numbers=para['line_numbers']
                )
                matches.append(match)
        matches.sort(key=lambda x: x.similarity_score, reverse=True)
        return matches[:max_results]
    def get_best_match(self, search_text: str) -> Optional[ParagraphMatch]:
        matches = self.find_paragraphs(search_text, max_results=1)
        return matches[0] if matches else None
    def search_multiple_terms(self, search_terms: List[str]) -> Dict[str, List[ParagraphMatch]]:
        results = {}
        for term in search_terms:
            results[term] = self.find_paragraphs(term)
        return results
    def get_context_around_match(self, match: ParagraphMatch, context_paragraphs: int = 1) -> List[str]:
        if not self.paragraphs:
            return [match.text]
        match_idx = None
        for i, para in enumerate(self.paragraphs):
            if (para['page_number'] == match.page_number and
                para['block_number'] == match.block_number):
                match_idx = i
                break
        if match_idx is None:
            return [match.text]
        start_idx = max(0, match_idx - context_paragraphs)
        end_idx = min(len(self.paragraphs), match_idx + context_paragraphs + 1)
        context_texts = []
        for i in range(start_idx, end_idx):
            prefix = ">>> " if i == match_idx else "    "
            context_texts.append(f"{prefix}{self.paragraphs[i]['text']}")
        return context_texts
    def print_match_details(self, match: ParagraphMatch, include_context: bool = False):
        print(f"Match found (Similarity: {match.similarity_score}%)")
        print(f"Page: {match.page_number}")
        print(f"Position: x={match.bbox[0]:.1f}, y={match.bbox[1]:.1f}")
        print(f"Size: {match.bbox[2]-match.bbox[0]:.1f}x{match.bbox[3]-match.bbox[1]:.1f}")
        print(f"Block: {match.block_number}")
        print("-" * 50)
        if include_context:
            context = self.get_context_around_match(match)
            for line in context:
                print(line)
        else:
            print(match.text)

In [6]:
def demo_usage():
    pdf_file = "median.pdf"
    try:
        with PDFParagraphLocator(pdf_file, min_similarity=60) as locator:
            paragraphs = locator.extract_paragraphs()
            print(f"Found {len(paragraphs)} paragraphs")
            search_queries = [
                """ By a comparison-based sorting algorithm, we mean an algorithm that learns about the elements of the
 input array A only by comparing them. Non-examples include radix sort, which needs to access the bits of A s
 elements, as well as bucket and counting sorts, which need to know the precise values of A s elements. (These
 algorithms run in linear time in certain cases, such as when all array elements are small integers.) Examples
 of comparison-based sorting algorithms include MergeSort, HeapSort, and QuickSort. Such algorithms are
 useful in practice because they are general-purpose and make no assumptions about what kind of elements
 you want to sort (e.g., the UNIX qsort function requires only a pointer to the data to be sorted and a
 function pointer to a user-speci ed comparison subroutine)."""
            ]

            print("\nSearching for paragraphs...")
            for query in search_queries:
                print(f"\nSearching for: '{query}'")
                matches = locator.find_paragraphs(query, max_results=3)
                if matches:
                    for i, match in enumerate(matches):
                        print(f"\nResult {i+1}:")
                        locator.print_match_details(match, include_context=True)
                else:
                    print("No matches found")
            sample_snippet = "The rapid advancement of technology has transformed"
            print(f"\nFinding best match for: '{sample_snippet}'")
            best_match = locator.get_best_match(sample_snippet)
            if best_match:
                print("Best match found:")
                locator.print_match_details(best_match, include_context=True)
            else:
                print("No suitable match found")

    except FileNotFoundError:
        print(f"Error: PDF file '{pdf_file}' not found")
        print("Please update the pdf_file variable with the correct path to your PDF")
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")

In [7]:
def interactive_search():
    """Interactive search function for user input"""
    pdf_file = input("Enter PDF file path: ").strip()

    try:
        with PDFParagraphLocator(pdf_file, min_similarity=70) as locator:
            paragraphs = locator.extract_paragraphs()
            print(f"Found {len(paragraphs)} paragraphs\n")
            while True:
                search_text = input("Enter text to search for (or 'quit' to exit): ").strip()
                if search_text.lower() == 'quit':
                    break
                matches = locator.find_paragraphs(search_text, max_results=5)
                if matches:
                    print(f"\nFound {len(matches)} matches:")
                    for i, match in enumerate(matches):
                        print(f"\n--- Match {i+1} ---")
                        locator.print_match_details(match)
                else:
                    print("No matches found")

    except FileNotFoundError:
        print(f"Error: PDF file '{pdf_file}' not found")
    except Exception as e:
        print(f"Error: {str(e)}")

In [8]:
def analyze_pdf_structure(pdf_path: str):
    try:
        with PDFParagraphLocator(pdf_path) as locator:
            paragraphs = locator.extract_paragraphs()
            """Basic statistics"""
            total_paragraphs = len(paragraphs)
            total_pages = len(locator.doc)
            char_counts = [p['char_count'] for p in paragraphs]
            word_counts = [p['word_count'] for p in paragraphs]
            avg_chars = sum(char_counts) / len(char_counts) if char_counts else 0
            avg_words = sum(word_counts) / len(word_counts) if word_counts else 0

            print(f"PDF Analysis Results:")
            print(f"Total pages: {total_pages}")
            print(f"Total paragraphs: {total_paragraphs}")
            print(f"Average paragraph length: {avg_chars:.1f} characters, {avg_words:.1f} words")
            print(f"Longest paragraph: {max(char_counts) if char_counts else 0} characters")
            print(f"Shortest paragraph: {min(char_counts) if char_counts else 0} characters")
            print(f"\nFirst 3 paragraphs:")
            for i, para in enumerate(paragraphs[:3]):
                print(f"\nParagraph {i+1} (Page {para['page_number']}):")
                print(f"Position: ({para['bbox'][0]:.1f}, {para['bbox'][1]:.1f})")
                print(f"Text: {para['text'][:100]}{'...' if len(para['text']) > 100 else ''}")

    except Exception as e:
        print(f"Error analyzing PDF: {str(e)}")

In [9]:
def search_for_keywords(pdf_path: str, keywords: List[str]):
    with PDFParagraphLocator(pdf_path, min_similarity=60) as locator:
        results = locator.search_multiple_terms(keywords)

        for keyword, matches in results.items():
            print(f"\nResults for '{keyword}':")
            if matches:
                for match in matches[:2]:
                    print(f"  Page {match.page_number} (Score: {match.similarity_score}%)")
                    print(f"  {match.text[:150]}...")
            else:
                print("  No matches found")

In [10]:
def export_matches_to_json(matches: List[ParagraphMatch], output_file: str):
    export_data = []
    for match in matches:
        export_data.append({
            'text': match.text,
            'page_number': match.page_number,
            'bbox': match.bbox,
            'similarity_score': match.similarity_score,
            'block_number': match.block_number,
            'line_numbers': match.line_numbers
        })

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    print(f"Exported {len(matches)} matches to {output_file}")

In [11]:
if __name__ == "__main__":
    PDF_FILE = "median.pdf"
    SEARCH_TEXTS = [
        """" By a comparison-based sorting algorithm, we mean an algorithm that learns about the elements of the
 input array A only by comparing them. Non-examples include radix sort, which needs to access the bits of A s
 elements, as well as bucket and counting sorts, which need to know the precise values of A s elements. (These
 algorithms run in linear time in certain cases, such as when all array elements are small integers.) Examples
 of comparison-based sorting algorithms include MergeSort, HeapSort, and QuickSort. Such algorithms are
 useful in practice because they are general-purpose and make no assumptions about what kind of elements
 you want to sort (e.g., the UNIX qsort function requires only a pointer to the data to be sorted and a
 function pointer to a user-speci ed comparison subroutine)."""
    ]

    try:
        analyze_pdf_structure(PDF_FILE)
        with PDFParagraphLocator(PDF_FILE, min_similarity=70) as locator:
            for search_text in SEARCH_TEXTS:
                matches = locator.find_paragraphs(search_text, max_results=2)
                if matches:
                    for match in matches:
                        locator.print_match_details(match)
                else:
                    print("No matches found")

    except FileNotFoundError:
        print(f"\nFile '{PDF_FILE}' not found. Please upload your PDF and update the file path.")
    except Exception as e:
        print(f"\nDemo error: {str(e)}")
        print("This is expected if you haven't uploaded a PDF file yet.")

PDF Analysis Results:
Total pages: 4
Total paragraphs: 34
Average paragraph length: 393.9 characters, 72.8 words
Longest paragraph: 2667 characters
Shortest paragraph: 24 characters

First 3 paragraphs:

Paragraph 1 (Page 1):
Position: (72.0, 72.9)
Text: CS161, Winter 2011 Handout #6

Paragraph 2 (Page 1):
Position: (91.4, 105.8)
Text: Notes on Linear-Time Selection, and a Sorting Lower Bound

Paragraph 3 (Page 1):
Position: (72.0, 163.8)
Text: In the Selection problem, we are given an array A containing n distinct numbers and an integer i ∈ {...
Match found (Similarity: 99%)
Page: 4
Position: x=72.0, y=266.8
Size: 468.1x213.2
Block: 10
--------------------------------------------------
Can the above linear-time bound be extended to the more general problem of sorting an array? We next show that the answer is “no” for comparison-based sorting algorithms. By a comparison-based sorting algorithm, we mean an algorithm that learns about the elements of the input array A only by comparing t